# import 

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output, State
from datetime import datetime
from dash import html, dcc, dash_table, callback_context
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import dashboard_util as du
# Using a Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SLATE])


strike_prices_nifty = [{'label': str(price), 'value': price} for price in range(18000, 24001, 50)]
strike_prices_banknifty = [{'label': str(price), 'value': price} for price in range(40000, 50000, 100)]
# df = pd.DataFrame(columns=['Order Type', 'Symbol', 'Price', 'Quantity', 'Stop Loss', 'Target'])
initial_columns =  [
    'tradingsymbol','quantity','average_price', 'ltp_last','pnl_calculated',
    
]
initial_columns_open = ['tradingsymbol', 'quantity', 'price', 'order_type']
df_open = pd.DataFrame(columns=initial_columns_open)
df = pd.DataFrame(columns=initial_columns)

app.layout = html.Div([
    html.H2("Akash 1 cliq", style={'textAlign': 'center'}),

    # Dropdowns and input row
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='symbol-dropdown',
            options=[{'label': 'NIFTY', 'value': 'NIFTY'}, {'label': 'BANKNIFTY', 'value': 'BANKNIFTY'}],
            value='NIFTY',
            placeholder="Select a symbol"), width=2),
        dbc.Col(dcc.DatePickerSingle(
            id='expiry-date-picker',
            min_date_allowed=datetime.today(),
            initial_visible_month=datetime.today(),
            placeholder='Select an expiry date'), width=3),
        dbc.Col(dcc.Dropdown(
            id='call-strike-dropdown',
            placeholder="Select Call Strike Price"), width=2),
        dbc.Col(dcc.Dropdown(
            id='put-strike-dropdown',
            placeholder="Select Put Strike Price"), width=2),
        dbc.Col(dcc.Input(
            id='lots-input',
            type='number',
            placeholder='Enter number of lots',
            min=1,
            value=1), width=2)
    ], className="mb-3"),

    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='order-type-dropdown',
            options=[{'label': 'MARKET', 'value': 'MARKET'}, {'label': 'LIMIT', 'value': 'LIMIT'}],
            value='MARKET',
            placeholder="Select order type"), width=2),
        dbc.Col(dcc.Input(
            id='price-input',
            type='number',
            placeholder='Enter price',
            min=0), width=2)
    ], className="mb-3"),

    # Price display row
    dbc.Row([
        dbc.Col(html.Div(id='call-price-display'), width=3, className="text-start"),
        dbc.Col(html.Div(id="nifty-display"), width=3, className="text-center"),
        dbc.Col(html.Div(id="bank-nifty-display"), width=3, className="text-start"),
        dbc.Col(html.Div(id='put-price-display'), width=3, className="text-end")
    ], className="mb-3"),

    # Buy/Sell buttons row
    dbc.Row([
        dbc.Col([
            html.Button("Buy Call", id='buy-call-button', n_clicks=0,
                        style={'backgroundColor': 'green', 'color': 'black', 'marginRight': '5px'}),
            html.Button("Sell Call", id='sell-call-button', n_clicks=0,
                        style={'backgroundColor': 'blue', 'color': 'black'})
        ], width=6, className="text-start"),
        dbc.Col([
            html.Button("Buy Put", id='buy-put-button', n_clicks=0,
                        style={'backgroundColor': 'green', 'color': 'black', 'marginRight': '5px'}),
            html.Button("Sell Put", id='sell-put-button', n_clicks=0,
                        style={'backgroundColor': 'blue', 'color': 'black'})
        ], width=6, className="text-end")
    ], className="mb-3"),

    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # in milliseconds
        n_intervals=0
    ),

    dcc.Interval(
        id='interval-component_minute',
        interval=1*90000,  # in milliseconds
        n_intervals=0
    ),

    html.Div(id='output-div'),  # Placeholder for callback output

    # MTM sum and centered buttons
    dbc.Row([
    dbc.Col(html.H4(id='mtm-sum', style={'textAlign': 'center'}), width='auto', className="d-flex justify-content-center"),
    dbc.Col(dcc.Input(id='stop-loss-input', type='number',debounce=True, placeholder='Set Stop-Loss Value'), width=2, className="d-flex justify-content-start"),
    dbc.Col(html.Button('Square Off', id='square-off-btn', n_clicks=0,className="me-2", style={'margin': '5px', 'width': '100px'}), width=2, className="d-flex justify-content-start")
    ], className="mb-3"),

    
    dbc.Row([
        dbc.Col(html.Button("Close Position", id="close-position-button", n_clicks=0, className="me-2", style={'margin': '5px', 'width': '120px'}), width='auto', className="d-flex justify-content-end"),
        dbc.Col(html.Button("Close Open", id="close-open-button", n_clicks=0, className="ms-2", style={'margin': '5px', 'width': '120px'}), width='auto', className="d-flex justify-content-start")
    ], justify="center", style={'margin': '20px'}),

    html.Div(id='output_close_position'),
    html.Div(id='output_close_open'),
    html.Div(id = "output_mtm_stoploss"),

    # Tabs containing the two tables
    dbc.Tabs([
        dbc.Tab(label='Positions', children=[
            html.H4("Trading Table"),
            dash_table.DataTable(
                id='trading-table',
                columns=[{"name": i, "id": i} for i in initial_columns],
                data=df.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'center', 'padding': '5px'},
                style_header={'backgroundColor': 'rgb(210, 210, 210)', 'color': 'black', 'fontWeight': 'bold'},
                style_cell_conditional=[{'if': {'column_id': c}, 'textAlign': 'left'} for c in ['Date', 'Region']],
                style_data={'color': 'black', 'backgroundColor': 'white'},
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(220, 220, 220)'},
                    {'if': {'column_id': 'pnl_calculated'}, 'fontWeight': 'bold', 'color': '#2ca02c'}
                ],
                page_size=10  # Set the number of rows to display
            ),
        ]),
        dbc.Tab(label='Open', children=[
            html.H4("Open Trades Table"),
            dash_table.DataTable(
                id='open-trades-table',
                columns=[{"name": col, "id": col} for col in initial_columns_open],
                data=df_open.to_dict('records'),
                editable=True,
                row_selectable="multi",
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left', 'padding': '5px'},
                style_header={'backgroundColor': 'rgb(210, 210, 210)', 'color': 'black', 'fontWeight': 'bold'},
                style_cell_conditional=[{'if': {'column_id': c}, 'textAlign': 'left'} for c in ['Date', 'Region']],
                style_data={'color': 'black', 'backgroundColor': 'white'},
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(220, 220, 220)'},
                    {'if': {'column_id': 'pnl_calculated'}, 'fontWeight': 'bold', 'color': '#2ca02c'}
                ],
                page_size=10  # Set the number of rows to display
            ),
            dbc.Row([
                dbc.Col([
                    dbc.Input(id="chg-quant", type="number", placeholder="Change Quantity", style={'marginBottom': '5px'}),
                    dbc.Input(id="chg-price", type="number", placeholder="Change Price", style={'marginBottom': '5px'}),
                    dbc.Button("Modify", id="modify-button", n_clicks=0, className="me-2"),
                    dbc.Button("Cancel", id="cancel-button", n_clicks=0)
                ], width=3)
            ]),
            html.Div(id='output-area')  # Placeholder for feedback
        ]),
        dbc.Tab(label='Executed', children=[
            html.H4("Executed Trades Table"),
            dash_table.DataTable(
                id='executed-trades-table',
                columns=[{"name": col, "id": col} for col in initial_columns_open + ["average_price","transaction_type",'status','order_timestamp']],
                data=df_open.to_dict('records'),
                editable=True,
                row_selectable="multi",
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left', 'padding': '5px'},
                style_header={'backgroundColor': 'rgb(210, 210, 210)', 'color': 'black', 'fontWeight': 'bold'},
                style_cell_conditional=[{'if': {'column_id': c}, 'textAlign': 'left'} for c in ['Date', 'Region']],
                style_data={'color': 'black', 'backgroundColor': 'white'},
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(220, 220, 220)'},
                    {'if': {'column_id': 'pnl_calculated'}, 'fontWeight': 'bold', 'color': '#2ca02c'}
                ],
                page_size=50  # Set the number of rows to display
            )
            ]),

        dbc.Tab(label='Active_OI', children=[
            html.H4("Active OI Table"),
            
            # Dropdown for selecting the minute interval
            html.Div([
                dcc.Dropdown(
                    id='minute-interval-dropdown',
                    options=[
                        {'label': 'Minute', 'value': 'minute'},
                        {'label': '3 Minutes', 'value': '3minute'},
                        {'label': '5 Minutes', 'value': '5minute'},
                        {'label': '15 Minutes', 'value': '15minute'}
                    ],
                    value='minute'  # Default value
                )
            ], style={'padding': '20px 0'}),
            
            dash_table.DataTable(
                id='active_oi_table',
                editable=False,
                style_table={'height': '300px', 'overflowY': 'auto'},
                style_cell={
                    'height': 'auto',
                    # all three widths are needed
                    'minWidth': '80px', 'width': '80px', 'maxWidth': '80px',
                    'whiteSpace': 'normal'
                },
                style_header={'backgroundColor': 'rgb(210, 210, 210)', 'color': 'black', 'fontWeight': 'bold'},
                # style_cell_conditional=[{'if': {'column_id': c}, 'textAlign': 'left'} for c in ['Date', 'Region']],
                style_data={'color': 'black', 'backgroundColor': 'white'},
                page_size=50  # Set the number of rows to display
            )
        ])
    ])
])

# kite order

In [ ]:
import random
import pandas as pd
from kite_trade import *
from enctoken import get_kite
kite = get_kite()

# inst = pd.DataFrame(kite.instruments("NFO"))
# inst_nse = pd.DataFrame(kite.instruments("NSE"))
# inst[inst.tradingsymbol == "NIFTY2432122000CE"]

In [ ]:
# df_pos_day = pd.DataFrame(kite.positions()['day'])
# df_pos_day[df_pos_day.quantity> 0]

In [ ]:
# open_cols = ['order_id','status','tradingsymbol','quantity','price','order_type']
# df_orders = pd.DataFrame(kite.orders())

In [ ]:
# df_orders

In [ ]:
# kite.modify_order(
#     variety=kite.VARIETY_REGULAR, 
#     order_id=order_id, 
#     parent_order_id=None, 
#     quantity= 50,
#     price=0.7,
#     order_type=kite.ORDER_TYPE_LIMIT, 
#     trigger_price=None,
#     validity=kite.VALIDITY_DAY,
#     disclosed_quantity=None,
# )
# kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=order_id)

In [ ]:
# It is not recommended to poll positions for updated P&L. You need to calculate that at your end using this formula
# pnl = (sellValue - buyValue) + (netQuantity * lastPrice * multiplier);

In [ ]:
# pos_col  = ['tradingsymbol','quantity','average_price','last_price','pnl','m2m','sell_value','buy_value','multiplier']
# df_positions = pd.DataFrame(kite.positions()['day'])[pos_col]
# df_positions = df_positions.merge(
#     pd.DataFrame(kite.ltp(("NFO:" + df_positions['tradingsymbol']).values)).T['last_price']
#     .reset_index()
#     .rename(columns = {'index':'tradingsymbol','last_price':"ltp_last"})
#     .assign(tradingsymbol = lambda x: x.tradingsymbol.str.split(':').str[1]),
#     on = 'tradingsymbol'
# )
# df_positions['pnl_calculated'] = (df_positions['sell_value'] - df_positions['buy_value']) + (df_positions['quantity'] * df_positions['ltp_last'] * df_positions['multiplier'])
# df_positions

In [ ]:
# import time
# while True:
#     print(pd.DataFrame(kite.positions()['net'])[pos_col])
#     time.sleep(5)

In [ ]:
# kite.margins()['equity']['available']['live_balance']
# kite.ltp("11895810")
# inst[inst.tradingsymbol.str.contains('NIFTY')].tradingsymbol.unique().tolist()
# orders = kite.orders()
# for i in orders:
#     print(i["status"])

In [ ]:
# # order_id = buy_sell_order(kite=kite,strike_price= 23100,ce_pe =  "CE", price = 1, trans_type = kite.TRANSACTION_TYPE_BUY, order_type =kite.ORDER_TYPE_LIMIT,quantity = 50)

# order_id = '240318501145968'
# kite.modify_order(
#     variety=kite.VARIETY_REGULAR, 
#     order_id=order_id, 
#     parent_order_id=None, 
#     quantity= 50,
#     price=0.7,
#     order_type=kite.ORDER_TYPE_LIMIT, 
#     trigger_price=None,
#     validity=kite.VALIDITY_DAY,
#     disclosed_quantity=None,
# )
# kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=order_id)

# call back 

In [ ]:

@app.callback(
    [Output('call-strike-dropdown', 'options'),
     Output('put-strike-dropdown', 'options')],
    Input('symbol-dropdown', 'value')
)
def update_strike_prices(selected_symbol):
    if selected_symbol == 'NIFTY':
        strike_prices = [{'label': str(price), 'value': price} for price in range(21000, 23001, 50)]
    elif selected_symbol == 'BANKNIFTY':
        strike_prices = [{'label': str(price), 'value': price} for price in range(45000, 49001, 100)]
    else:
        return [], []

    return strike_prices, strike_prices


@app.callback(
    [
     Output('nifty-display', 'children'),
     Output('bank-nifty-display', 'children')],
    [Input('interval-component', 'n_intervals')],
)
def update_nifty_bn_prices(n):
    ltp_dict = kite.ltp([256265, 260105])
    nifty_price = ltp_dict["256265"]["last_price"]
    banknifty_price = ltp_dict["260105"]["last_price"]
    nifty_display = f"Nifty Price: {nifty_price}" if nifty_price is not None else "Select a nifty "
    bank_nifty_display = f"BankNifty Price: {banknifty_price}" if banknifty_price is not None else "Select a bn"

    return  nifty_display, bank_nifty_display


@app.callback(
    [Output('call-price-display', 'children'),
     Output('put-price-display', 'children')],
    [Input('interval-component', 'n_intervals')],
    [State('call-strike-dropdown', 'value'),
     State('put-strike-dropdown', 'value'),
     State('symbol-dropdown', 'value'),]
)
def update_prices(n, call_strike, put_strike,symbol):
    symbol_ce = 256265
    symbol_pe = 256265
    call_price = None
    put_price = None
    if symbol == 'NIFTY':
        if call_strike is not None:
          symbol_ce = f"NFO:NIFTY24410{call_strike}CE"
        if put_strike is not None:
            symbol_pe = f"NFO:NIFTY24410{put_strike}PE"
    elif symbol == 'BANKNIFTY':
        if call_strike is not None:
            symbol_ce = f"NFO:BANKNIFTY24410{call_strike}CE"
        if put_strike is not None:
            symbol_pe = f"NFO:BANKNIFTY24410{put_strike}PE"

    if call_strike is not None and put_strike is not None:
        ltp_dict = kite.ltp([symbol_ce, symbol_pe])
        call_price = ltp_dict[symbol_ce]['last_price']
        put_price = ltp_dict[symbol_pe]['last_price']

    call_display = f"Call Price: {call_price}" if call_price is not None else "Select a Call Strike"
    put_display = f"Put Price: {put_price}" if put_price is not None else "Select a Put Strike"

    return call_display, put_display



@app.callback(
    Output('output-div', 'children'),
    [Input('buy-call-button', 'n_clicks'), 
     Input('sell-call-button', 'n_clicks'),
     Input('buy-put-button', 'n_clicks'),
     Input('sell-put-button', 'n_clicks')],
    [State('symbol-dropdown', 'value'),
     State('expiry-date-picker', 'date'),
     State('call-strike-dropdown', 'value'),
     State('put-strike-dropdown', 'value'),
     State('lots-input', 'value'),
     State('order-type-dropdown', 'value'),
     State('price-input', 'value')],
    prevent_initial_call=True
)
def update_output(buy_call, sell_call, buy_put, sell_put, symbol, expiry_date, call_strike, put_strike, lots, order_type, price):
    ctx = dash.callback_context
    
    # Symbol
    if symbol == "NIFTY":
        call_strike = f"NIFTY24410{call_strike}"
        put_strike = f"NIFTY24410{put_strike}"
        quantity = lots * 50
    else:
        call_strike = f"BANKNIFTY24410{call_strike}"
        put_strike = f"BANKNIFTY24410{put_strike}"
        quantity = lots * 15

    # order type 
    if order_type == 'LIMIT':
        order_type = kite.ORDER_TYPE_LIMIT
    else:
        order_type = kite.ORDER_TYPE_MARKET
        price = None



    
    if not ctx.triggered:
        return "No order executed"
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        order_details = f"Symbol: {symbol}, Expiry: {expiry_date}, Call Strike: {call_strike}, Put Strike: {put_strike}, Lots: {lots}, Order Type: {order_type}, Price: {price}"

        if button_id == 'buy-call-button':
            order_id = du.buy_sell_order(kite=kite,strike_price= call_strike,ce_pe =  "CE",
                                        price = price, trans_type = kite.TRANSACTION_TYPE_BUY,
                                        order_type =order_type,quantity = quantity)
            return f"Executed Buy Call order. {order_details}"
        elif button_id == 'sell-call-button':
            order_id = du.buy_sell_order(kite=kite,strike_price= call_strike,ce_pe =  "CE",
                                        price = price, trans_type = kite.TRANSACTION_TYPE_SELL,
                                        order_type =order_type,quantity = quantity)
            return f"Executed Sell Call order. {order_details}"
        elif button_id == 'buy-put-button':
            order_id = du.buy_sell_order(kite=kite,strike_price= put_strike,ce_pe =  "PE",
                                        price = price, trans_type = kite.TRANSACTION_TYPE_BUY,
                                        order_type =order_type,quantity = quantity)
            return f"Executed Buy Put order. {order_details}"
        elif button_id == 'sell-put-button':
            order_id = du.buy_sell_order(kite=kite,strike_price= put_strike,ce_pe =  "PE",
                                        price = price, trans_type = kite.TRANSACTION_TYPE_SELL,
                                        order_type =order_type,quantity = quantity)
            return f"Executed Sell Put order. {order_details}"


@app.callback(
    Output('trading-table', 'data'),
    Input('interval-component', 'n_intervals')
)
def update_table_postion(n):
    # This function will fetch new data; here we just generate random data for demonstration
    pos_col  = ['tradingsymbol','quantity','average_price','last_price','pnl','m2m','sell_value','buy_value','multiplier']
    df_positions = pd.DataFrame(kite.positions()['day'])[pos_col]
    df_positions = df_positions.merge(
        pd.DataFrame(kite.ltp(("NFO:" + df_positions['tradingsymbol']).values)).T['last_price']
        .reset_index()
        .rename(columns = {'index':'tradingsymbol','last_price':"ltp_last"})
        .assign(tradingsymbol = lambda x: x.tradingsymbol.str.split(':').str[1]),
        on = 'tradingsymbol'
    )
    df_positions['pnl_calculated'] = (df_positions['sell_value'] - df_positions['buy_value']) + (df_positions['quantity'] * df_positions['ltp_last'] * df_positions['multiplier'])
    df_positions =  df_positions.round(2)
    # append 5 empty rows to the table
    df_positions = df_positions.append([{}]*3, ignore_index=True)
    return df_positions.to_dict('records')


@app.callback(
    [Output('mtm-sum', 'children'),
     Output('mtm-sum', 'style'),
     Output('output_mtm_stoploss', 'children')],
    [Input('interval-component', 'n_intervals'),
     Input('square-off-btn', 'n_clicks')],
    [State('trading-table', 'data'),
     State('stop-loss-input', 'value')],
    prevent_initial_call=True
)
def update_mtm(n_intervals, n_clicks, data, stop_loss):
    pos_txt = None
    df_updated = pd.DataFrame(data)
    mtm_sum = df_updated['pnl_calculated'].sum()
    mtm_text = f"MTM: {mtm_sum:.2f}"

    color = 'green' if mtm_sum > 0 else 'orange' if mtm_sum < 0 else 'black'
    style = {'textAlign': 'center', 'marginRight': '20px', 'display': 'inline-block', 'color': color}

    # Check for button click and if MTM is below stop-loss
    if n_clicks > 0 and stop_loss is not None and mtm_sum < stop_loss:
        for row in data:
            if row['quantity'] > 0:
                ce_pe = row['tradingsymbol'][-2:]
                strike_price = row['tradingsymbol'][:-2]
                order_id = du.buy_sell_order(kite=kite,strike_price= strike_price,ce_pe =  ce_pe,
                                        price = None, trans_type = kite.TRANSACTION_TYPE_SELL,
                                        order_type =kite.ORDER_TYPE_MARKET,quantity = row['quantity'])
                
        pos_txt = f"All Positions squared off successfully {stop_loss}"
    return mtm_text,style,pos_txt


@app.callback(
    Output('open-trades-table', 'data'),
    Input('interval-component', 'n_intervals')
)
def update_table_open(n):
    # Update this function to fetch new data
    open_cols = ['order_id','status','tradingsymbol','quantity','price','order_type']
    data = pd.DataFrame(kite.orders()).query("status == 'OPEN'")[open_cols]
    return data.to_dict('records')


@app.callback(
    Output('executed-trades-table', 'data'),
    Input('interval-component', 'n_intervals')
)
def update_table_executed(n):
    # Update this function to fetch new data
    open_cols = ['order_id','status','tradingsymbol','quantity',"transaction_type","average_price",'order_type','order_timestamp']
    data = pd.DataFrame(kite.orders()).sort_values("order_timestamp", ascending=False)[open_cols]
    data = data.query("status != 'CANCELLED'")
    return data.to_dict('records')


@app.callback(
    Output('output-area', 'children'),
    Input('modify-button', 'n_clicks'),
    Input('cancel-button', 'n_clicks'),
    State('open-trades-table', 'selected_rows'),
    State('open-trades-table', 'data'),
    State('chg-quant', 'value'),
    State('chg-price', 'value'),
    prevent_initial_call=True
)
def perform_operation(modify_clicks, cancel_clicks, selected_rows, rows_data,chg_quant, chg_price):
    ctx = dash.callback_context

    if not selected_rows:
        return "No rows selected!"

    if ctx.triggered:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        selected_data = [rows_data[i] for i in selected_rows]  # Data of selected rows

        if button_id == "modify-button":
            kite.modify_order(
                variety=kite.VARIETY_REGULAR, 
                order_id=selected_data[0]['order_id'], 
                parent_order_id=None, 
                quantity= chg_quant,
                price=chg_price,
                order_type=kite.ORDER_TYPE_LIMIT, 
                trigger_price=None,
                validity=kite.VALIDITY_DAY,
                disclosed_quantity=None,
            )
            return f"Modify clicked: {selected_data}"
        elif button_id == "cancel-button":
            kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=selected_data[0]['order_id'])
            return f"Cancel clicked: {selected_data}"
    return ""



@app.callback(
    Output('output_close_position', 'children'),
    Input('close-position-button', 'n_clicks'),
    State('trading-table', 'data'),
    prevent_initial_call=True
)
def close_position(n_clicks, rows):
    if n_clicks > 0:
        for row in rows:
            if row['quantity'] > 0:
                ce_pe = row['tradingsymbol'][-2:]
                strike_price = row['tradingsymbol'][:-2]
                order_id = du.buy_sell_order(kite=kite,strike_price= strike_price,ce_pe =  ce_pe,
                                        price = None, trans_type = kite.TRANSACTION_TYPE_SELL,
                                        order_type =kite.ORDER_TYPE_MARKET,quantity = row['quantity'])
                
                print(f"Squaring off position for {row['tradingsymbol']}")
            

    return "All Positions squared off successfully"


@app.callback(
    Output('output_close_open', 'children'),
    Input('close-open-button', 'n_clicks'),
    prevent_initial_call=True
)
def close_open(n_clicks):
    if n_clicks > 0:
        open_cols = ['order_id','status','tradingsymbol','quantity','price','order_type']
        data = pd.DataFrame(kite.orders()).query("status == 'OPEN'")[open_cols]
        for row in data.to_dict('records'):
            kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=row['order_id'])
    return "Open orders canceled."




import datetime
expiry_sp = 'BANKNIFTY24410'
start_dt = datetime.datetime.now().strftime("%Y-%m-%d")
end_dt = datetime.datetime.now().strftime("%Y-%m-%d")
time_frame = "minute"

inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']



@app.callback(
    [Output('active_oi_table', 'columns'),
     Output('active_oi_table', 'data'),
     Output('active_oi_table', 'style_data_conditional'),
     Output('active_oi_table', 'style_cell_conditional')],
    Input('interval-component_minute', 'n_intervals'),
    Input('minute-interval-dropdown', 'value')
)
def update_active_oi(n, time_frame):
    # Update this function to fetch new data
    time_frame= time_frame
    data = du.active_oi_chg(kite,expiry_march, start_dt, end_dt, time_frame=time_frame)
    # columns = [{"name": i, "id": i, 'type': 'numeric', 'format': {'specifier': ','}} for i in data.columns]
    columns = [{"name": i, "id": i} for i in data.columns]
     # Dynamically generate styles based on the updated columns
    style_data_conditional = [
        {'if': {'column_id': c['id']}, 'backgroundColor': '#FFDDC1'} for c in columns if c['id'].startswith('oi_')
    ] + [
        {'if': {'column_id': c['id']}, 'backgroundColor': '#C1FFD7'} for c in columns if c['id'].startswith('chg')
    ] 
    
    strike_select = set([c.split("_")[1] for c in data.columns[1:]])
    strike_groups = {}
    
    for strike in strike_select:
        strike_groups[strike] = [c for c in data.columns[1:] if c.split("_")[1] == strike]
    
    style_cell_conditional = []
    for strike, col in strike_groups.items():
        if columns:
            # Left border for the first column in the group
            style_cell_conditional.append({
                'if': {'column_id': col[0]},
                'borderLeft': '3px solid black'
            })
            # Right border for the last column in the group
            style_cell_conditional.append({
                'if': {'column_id': col[-1]},
                'borderRight': '3px solid black'
            })
    
    data = du.format_dataframe(data)
    return columns, data.head(20).to_dict('records'),style_data_conditional, style_cell_conditional


# Run

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, port = 1222,jupyter_mode="external")

Dash app running on http://127.0.0.1:1222/


In [ ]:
xx

NameError: name 'xx' is not defined

48900 [49000] [48800]
2024-04-09 2024-04-09
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
c:\Users\akash.verma\Miniconda3\envs\py37\lib\site-packages\pandas\core\indexes\base.py in get_loc(
    self=Index([], dtype='object'),
    key='pnl_calculated',
    method=None,
    tolerance=None
)
   3360             try:
-> 3361                 return self._engine.get_loc(casted_key)
        self._engine.get_loc = <built-in method get_loc of pandas._libs.index.ObjectEngine object at 0x000001D836E2E948>
        casted_key = 'pnl_calculated'
   3362             except KeyError as err:

c:\Users\akash.verma\Miniconda3\envs\py37\lib\site-packages\pandas\_libs\index.pyx in pandas._libs.index.IndexEngine.get_loc()

c:\Users\akash.verma\Miniconda3\envs\py37\lib\site-packages\pandas\_libs\index.pyx in pandas._libs.index.IndexEngine.get_loc()

pandas\_libs\hashtable_class_helper.pxi in pandas._libs

In [ ]:
# # First, install dash if you haven't already
# # !pip install dash

# import dash
# from dash import html, dcc, Input, Output
# import dash_bootstrap_components as dbc

# # Initialize the Dash application
# app = dash.Dash(__name__)

# # Define the layout of the app
# app.layout = dbc.Container([
#     dbc.Row([
#         dbc.Col(dcc.Dropdown(id='broker-name', options=[{'label': 'Trade smart (User: YT****30)', 'value': 'Trade smart'}], value='Trade smart', placeholder="Broker Name"), width=3),
#         dbc.Col(dcc.Dropdown(id='exchange', options=[{'label': 'NSE', 'value': 'NSE'}], value='NSE', placeholder="Exchange"), width=3),
#         dbc.Col(dcc.Dropdown(id='segment', options=[{'label': 'Options', 'value': 'Options'}], value='Options', placeholder="Segment"), width=3),
#         dbc.Col(dcc.Dropdown(id='symbol', options=[{'label': 'NIFTY', 'value': 'NIFTY'}], value='NIFTY', placeholder="Symbol"), width=3),
#     ], className="mb-2"),
#     dbc.Row([
#         dbc.Col(dcc.Dropdown(id='date', options=[{'label': 'Thu, Sep 07,2023', 'value': '2023-09-07'}], value='2023-09-07', placeholder="Date"), width=3),
#         dbc.Col(dcc.Dropdown(id='expiry-date', options=[{'label': '07-Sep-2023', 'value': '2023-09-07'}], value='2023-09-07', placeholder="Expiry Date"), width=3),
#         dbc.Col(dcc.Dropdown(id='call-strike', options=[{'label': '19700', 'value': '19700'}], value='19700', placeholder="Call Strike"), width=3),
#         dbc.Col(dcc.Dropdown(id='put-strike', options=[{'label': '19850', 'value': '19850'}], value='19850', placeholder="Put Strike"), width=3),
#     ], className="mb-2"),
#     dbc.Row([
#         dbc.Col(dcc.Dropdown(id='lots', options=[{'label': '5 (Lot size:250)', 'value': '5'}], value='5', placeholder="Lots"), width=3),
#         dbc.Col(html.Button('Show Options List', id='show-options-list', n_clicks=0, className="btn btn-primary"), width=3),
#         dbc.Col(html.Button('Buy Call', id='buy-call', n_clicks=0, className="btn btn-success"), width=3),
#         dbc.Col(html.Button('Sell Call', id='sell-call', n_clicks=0, className="btn btn-danger"), width=3),
#     ], className="mb-2"),
#     dbc.Row([
#         dbc.Col(html.Button('Buy Put', id='buy-put', n_clicks=0, className="btn btn-success"), width=3),
#         dbc.Col(html.Button('Sell Put', id='sell-put', n_clicks=0, className="btn btn-danger"), width=3),
#         dbc.Col(html.Button('Close All Positions/F6', id='close-all-positions', n_clicks=0, className="btn btn-warning"), width=3),
#         dbc.Col(html.Button('Cancel All Orders / F7', id='cancel-all-orders', n_clicks=0, className="btn btn-secondary"), width=3),
#     ], className="mb-2"),
#     dbc.Row([
#         dbc.Col(html.Div(id='output-container'), width=12),
#     ])
# ], fluid=True)


# # Define callback to update the output container
# @app.callback(
#     Output('output-container', 'children'),
#     [Input('show-options-list', 'n_clicks'),
#      Input('buy-call', 'n_clicks'),
#      Input('sell-call', 'n_clicks'),
#      Input('buy-put', 'n_clicks'),
#      Input('sell-put', 'n_clicks'),
#      Input('close-all-positions', 'n_clicks'),
#      Input('cancel-all-orders', 'n_clicks')]
# )
# def update_output(show_options_list, buy_call, sell_call, buy_put, sell_put, close_all_positions, cancel_all_orders):
#     # Update the output based on which button was clicked
#     # For demonstration purposes, we'll just return the button name and number of clicks
#     ctx = dash.callback_context
#     if not ctx.triggered:
#         button_id = 'No clicks yet'
#     else:
#         button_id = ctx.triggered[0]['prop_id'].split('.')[0]

#     return html.Div(f"{button_id} clicked")

# # Run the app
# if __name__ == '__main__':
#     app.run_server(debug=True, port = 1222)


In [ ]:
# from jupyter_dash import JupyterDash
# from dash import dcc, html
# import dash_bootstrap_components as dbc
# from dash.dependencies import Input, Output

# final dash

In [ ]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Order Place", href="/order_place")),
        dbc.NavItem(dbc.NavLink("OI table", href="/oi_table")),
    ],
    brand="Akash Trading Paradise",
    brand_href="/",
    color="black",
    dark=True,
)

content = html.Div(id="page-content")

contentHome = html.P("Welcome to the Trading Paradise!")

contentMaster = html.Div(id="master-content0", children=[
    dbc.Row([
        dbc.Col(dcc.Dropdown(id='exchange-dropdown', options=[{'label': i, 'value': i} for i in ['NSE', 'BSE', 'NFO']], placeholder="Select Exchange")),
        dbc.Col(dcc.Dropdown(id='symbol-dropdown', options=[{'label': i, 'value': i} for i in ['NIFTY', 'BANKNIFTY']], placeholder="Select Symbol")),
        dbc.Col(dcc.DatePickerSingle(id='trade-date', placeholder="Select Date")),
        dbc.Col(dcc.DatePickerSingle(id='expiry-date-input', placeholder="Select Expiry Date")),
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Input(id='call-strike-input', type='number', placeholder="Call Strike")),
        dbc.Col(dcc.Input(id='put-strike-input', type='number', placeholder="Put Strike")),
        dbc.Col(dcc.Dropdown(id='lot-size-dropdown', options=[{'label': i, 'value': i} for i in range(1, 21)], placeholder="Select Lot Size")),
        dbc.Col(dcc.Dropdown(id='order-type-dropdown', options=[{'label': i, 'value': i} for i in ['MARKET', 'LIMIT']], placeholder="Select Order Type")),
    ]),
    dbc.Row([
        dbc.Col(html.Button('Buy Call', id='buy-call-button', n_clicks=0)),
        dbc.Col(html.Button('Sell Call', id='sell-call-button', n_clicks=0)),
        dbc.Col(html.Button('Buy Put', id='buy-put-button', n_clicks=0)),
        dbc.Col(html.Button('Sell Put', id='sell-put-button', n_clicks=0)),
    ]),
])



app.layout = dbc.Container([dcc.Location(id="url"), navbar, content], fluid=True)

@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return contentHome
    elif pathname == "/order_place":
        return contentMaster
    elif pathname == "/oi_table":
        # Placeholder for OI Table page content
        return html.P("OI Table Placeholder")
    # Return a 404 message when the page is not found
    return html.Div([
        html.H1("404: Not found", className="text-danger"),
        html.Hr(),
        html.P(f"The pathname {pathname} was not recognised..."),
    ], className="p-3 bg-light rounded-3")

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True, port = 1222,jupyter_mode="external")


Dash app running on http://127.0.0.1:1222/


In [ ]:
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP],suppress_callback_exceptions=True)
navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Order Place", href="/order_place")),
        dbc.NavItem(dbc.NavLink("OI table", href="/oi_table")),
        # dbc.NavItem(dbc.NavLink("Demand Browser", href="/demand")),
        # dbc.NavItem(dbc.NavLink("Demand Model Browser", href="/demandmodel")),
        # dbc.NavItem(dbc.NavLink("Metrics", href="/metrics")),
        # dbc.NavItem(dbc.NavLink("Elasticity", href="/elasticity")),
    ],
    brand="Akash Tradeing paradise",
    brand_href="/",
    color="black",
    dark=True
)
content = html.Div(id="page-content")
content404 = html.Div(
    [
        html.H1("404: Not found", className="text-danger"),
        html.Hr(),
        html.P(f"The pathname was not recognised..."),
    ],
    className="p-3 bg-light rounded-3",
)
contentHome = html.P("Welcome to the Trading paradise!")

contentMaster = html.Div(id="master-content0",children=[
dbc.Row([
        dbc.Col(dcc.Dropdown(id='exchange-dropdown', 
                             options=[{'label': i, 'value': i} for i in ['NSE', 'BSE', 'NFO']], 
                             placeholder="Select Exchange")),
        dbc.Col(dcc.Dropdown(id='symbol-dropdown', 
                             options=[{'label': i, 'value': i} for i in ['NIFTY', 'BANKNIFTY']], 
                             placeholder="Select Symbol")),
        dbc.Col(dcc.Input(id='date-input', type='date', placeholder="Select Date")),
        dbc.Col(dcc.Input(id='expiry-date-input', type='date', placeholder="Select Expiry Date")),
    ]),
    dbc.Row([
        dbc.Col(dcc.Input(id='call-strike-input', type='number', placeholder="Call Strike")),
        dbc.Col(dcc.Input(id='put-strike-input', type='number', placeholder="Put Strike")),
        dbc.Col(dcc.Dropdown(id='lot-size-dropdown', 
                             options=[{'label': i, 'value': i} for i in range(1, 21)], 
                             placeholder="Select Lot Size")),
        dbc.Col(dcc.Dropdown(id='order-type-dropdown', 
                             options=[{'label': i, 'value': i} for i in ['MARKET', 'LIMIT']], 
                             placeholder="Select Order Type")),
    ]),
    dbc.Row([
        dbc.Col(html.Button('Buy Call', id='buy-call-button', n_clicks=0)),
        dbc.Col(html.Button('Sell Call', id='sell-call-button', n_clicks=0)),
        dbc.Col(html.Button('Buy Put', id='buy-put-button', n_clicks=0)),
        dbc.Col(html.Button('Sell Put', id='sell-put-button', n_clicks=0)),
    ]),

])


# Callback example (you can expand this with actual logic)
@app.callback(
    Output('navigation-layout', 'children'),
    Input('buy-call-button', 'n_clicks'),
    prevent_initial_call=True
)
def update_output(n_clicks):
    if n_clicks > 0:
        return f'Buy Call button clicked {n_clicks} times'
    return 'Navigation Layout Placeholder'

app.layout = dbc.Container([dcc.Location(id="url"),navbar,content])

@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return contentHome
    elif pathname == "/order_place":
        return contentMaster
    elif pathname == "/oi_table":
        return None
    
    # elif pathname == "/demand":
    #     return contentDemand
    # elif pathname == "/demandmodel":
    #     return contentModel
    # elif pathname == "/metrics":
    #     return contentMetrics
    # elif pathname == "/elasticity":
    #     return contentElasticity
    
    return content404

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, port=1222)


In [ ]:
xx

NameError: name 'xx' is not defined

In [ ]:

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Option Trading Dashboard"),
    html.Label("Exchange:"),
    dcc.Dropdown(
        id='exchange-dropdown',
        options=[
            {'label': 'NSE', 'value': 'NSE'},
            {'label': 'BSE', 'value': 'BSE'},
            {'label': 'NFO', 'value': 'NFO'},
        ],
        value='NSE'  # Default value
    ),
    html.Label("Symbol:"),
    dcc.Dropdown(
        id='symbol-dropdown',
        options=[
            {'label': 'NIFTY', 'value': 'NIFTY'},
            {'label': 'BANKNIFTY', 'value': 'BANKNIFTY'},
        ],
        value='NIFTY'  # Default value
    ),
    html.Label("Date:"),
    dcc.DatePickerSingle(
        id='trade-date',
    ),
    html.Label("Expiry Date:"),
    dcc.DatePickerSingle(
        id='expiry-date',
    ),
    html.Label("Call Strike:"),
    dcc.Input(id='call-strike', type='number', value=0),
    html.Label("Put Strike:"),
    dcc.Input(id='put-strike', type='number', value=0),
    html.Label("Lot Size:"),
    dcc.Slider(
        id='lot-size',
        min=1,
        max=20,
        step=1,
        value=1,
        marks={i: str(i) for i in range(1, 21)},
    ),
    html.Label("Order Type:"),
    dcc.Dropdown(
        id='order-type-dropdown',
        options=[
            {'label': 'MARKET', 'value': 'MARKET'},
            {'label': 'LIMIT', 'value': 'LIMIT'},
        ],
        value='MARKET'
    ),
    html.Button('Buy Call', id='buy-call-btn', n_clicks=0),
    html.Button('Sell Call', id='sell-call-btn', n_clicks=0),
    html.Button('Buy Put', id='buy-put-btn', n_clicks=0),
    html.Button('Sell Put', id='sell-put-btn', n_clicks=0),
    html.Div(id='container-button-basic')
])


@app.callback(
    Output('container-button-basic', 'children'),
    Input('buy-call-btn', 'n_clicks'),
    Input('sell-call-btn', 'n_clicks'),
    Input('buy-put-btn', 'n_clicks'),
    Input('sell-put-btn', 'n_clicks')
)
def display_click(btn1, btn2, btn3, btn4):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = 'No clicks yet'
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'buy-call-btn':
        return 'Buy Call clicked'
    elif button_id == 'sell-call-btn':
        return 'Sell Call clicked'
    elif button_id == 'buy-put-btn':
        return 'Buy Put clicked'
    elif button_id == 'sell-put-btn':
        return 'Sell Put clicked'
    else:
        return 'No clicks yet'

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, port=1222)


In [ ]:
import dash
from dash import dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import pandas as pd  # Assuming your OI data can be loaded or generated into a DataFrame

# Sample data for the OI Data table
# Replace this with your actual data loading mechanism
oi_data = pd.DataFrame({
    'Contract': ['NIFTY21SEP15000CE', 'NIFTY21SEP15000PE'],
    'Open Interest': [10000, 20000],
    'Change in OI': [500, -500],
    'Volume': [7500, 5600]
})

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP],suppress_callback_exceptions=True)

# Define the layout for the "Trading" page
def serve_trading_page():
    return html.Div([
        html.H1("Option Trading Dashboard"),
        # Your existing components for trading
        # Add your trading dashboard components here
        html.Hr(),
        dbc.Button("Go to OI Data", href="/oi-data"),
    ])

# Define the layout for the "OI Data" page
def serve_oi_data_page():
    return html.Div([
        html.H1("OI Data"),
        dbc.Table.from_dataframe(oi_data, striped=True, bordered=True, hover=True),
        html.Hr(),
        dbc.Button("Back to Trading", href="/"),
    ])

# Update the index page to handle multi-page app routing
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/oi-data':
        return serve_oi_data_page()
    else:
        return serve_trading_page()

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

if __name__ == '__main__':
    app.run_server(debug=True, port=1222)


In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Initialize the JupyterDash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Row([
        dbc.Col(dcc.Dropdown(id='exchange-dropdown', 
                             options=[{'label': i, 'value': i} for i in ['NSE', 'BSE', 'NFO']], 
                             placeholder="Select Exchange")),
        dbc.Col(dcc.Dropdown(id='symbol-dropdown', 
                             options=[{'label': i, 'value': i} for i in ['NIFTY', 'BANKNIFTY']], 
                             placeholder="Select Symbol")),
        dbc.Col(dcc.Input(id='date-input', type='date', placeholder="Select Date")),
        dbc.Col(dcc.Input(id='expiry-date-input', type='date', placeholder="Select Expiry Date")),
    ]),
    dbc.Row([
        dbc.Col(dcc.Input(id='call-strike-input', type='number', placeholder="Call Strike")),
        dbc.Col(dcc.Input(id='put-strike-input', type='number', placeholder="Put Strike")),
        dbc.Col(dcc.Dropdown(id='lot-size-dropdown', 
                             options=[{'label': i, 'value': i} for i in range(1, 21)], 
                             placeholder="Select Lot Size")),
        dbc.Col(dcc.Dropdown(id='order-type-dropdown', 
                             options=[{'label': i, 'value': i} for i in ['MARKET', 'LIMIT']], 
                             placeholder="Select Order Type")),
    ]),
    dbc.Row([
        dbc.Col(html.Button('Buy Call', id='buy-call-button', n_clicks=0)),
        dbc.Col(html.Button('Sell Call', id='sell-call-button', n_clicks=0)),
        dbc.Col(html.Button('Buy Put', id='buy-put-button', n_clicks=0)),
        dbc.Col(html.Button('Sell Put', id='sell-put-button', n_clicks=0)),
    ]),
    html.Div(id='navigation-layout', children=[
        # Placeholder for navigation layout - you can add more components or callbacks here
        html.H3('Navigation Layout Placeholder'),
    ]),
    html.Div(id='oi-layout', children=[
        # Placeholder for OI layout - you can add more components or callbacks here
        html.H3('OI Layout Placeholder'),
    ]),
])

# Callback example (you can expand this with actual logic)
@app.callback(
    Output('navigation-layout', 'children'),
    Input('buy-call-button', 'n_clicks'),
    prevent_initial_call=True
)
def update_output(n_clicks):
    if n_clicks > 0:
        return f'Buy Call button clicked {n_clicks} times'
    return 'Navigation Layout Placeholder'

# Run the app
app.run_server( debug = True, port = 1222)


TypeError: an integer is required (got type NoneType)